# 2.2. Dask Delayed Objects

When called, every `delayed` function returns a `Delayed` object.  Each `Delayed` object represents a node in a *task graph*, and each `Delayed` object gives you the ability to examine and visualize the *task graph* that leads up to that node in the graph.

![](inc-add.svg)

In [ ]:
import sleeplock
import dask

## Example: *Delayed Python, Again*

Let's redefine the same `inc` and `dec` operators as in the previous example, but this time we'll use `dask.delayed` as a decorator.

In [ ]:
@dask.delayed
def inc(i):
    sleeplock.sleep(i)
    return i + 1

In [ ]:
@dask.delayed
def dec(i):
    sleeplock.sleep(i)
    return i - 1

In [ ]:
%time i_2 = inc(2)
i_2

In [ ]:
%time d_i_2 = dec(i_2)
d_i_2

## Delayed.compute()

To force the `delayed` functions to compute and return the result, we call the `compute` method of the `Delayed` object.

In [ ]:
%time i_2.compute()

In [ ]:
%time d_i_2.compute()

### Notice!

The computation of `i2` took 5 seconds, which is the time required to compute `inc(2)` plus the time required to compute `dec(3)`!

But we already computed `i1`, so why are we computing it, again?

### NOTE:

In addition to using the `compute` method of a Delayed object, you can also compute a `Delayed` object with the `compute` function in Dask.

In [ ]:
%time x_i_2, x_d_i_2 = dask.compute(i_2, d_i_2)
x_i_2, x_d_i_2

### Notice!

Did you notice that this computed both `Delayed` objects at the same time (in parallel)?

## Delayed.persist()

To keep the computed result of a `Delayed` object in memory, so that it is available later, we use the `persist` method of the `Delayed` objects.

In [ ]:
%time i_2_p = inc(2).persist()
i_2_p

In [ ]:
%time i_2_p.compute()

In [ ]:
%time d_i_2_p = dec(i_2_p)
d_i_2_p

In [ ]:
%time d_i_2_p.compute()

### Notice!

Now, the computation of `i2` only took as long as it took to compute `dec(3)` because the result of `i1` was persisted in memory.

### NOTE:

Like the `dask.compute` function, you can also persist `Delayed` objects with the `dask.persist` function:

In [ ]:
%time p_i_2, p_d_i_2 = dask.persist(i_2, d_i_2)
p_i_2, p_d_i_2

## Delayed.key

Each `Delayed` object has a unique identifier, called a `key`, which can be returned with the `key` attribute.

In [ ]:
i_2.key

In [ ]:
i_2_p.key

In [ ]:
d_i_2.key

In [ ]:
d_i_2_p.key

## Delayed.dask

These `key`s are used to uniquely identify each task in a *Task Graph*, and the *Task Graph* can be viewed as dictionary-like object associated with the `dask` attribute of the `Delayed` object.

In [ ]:
# Short function to print out a Task Graph
def print_dask(dobj):
    for key in dobj.dask:
        print('{}:'.format(key))
        if isinstance(dobj.dask[key], tuple):
            print('    function:  {}'.format(dobj.dask[key][0]))
            print('    arguments: {}'.format(dobj.dask[key][1:]))
        else:
            print('    value: {}'.format(dobj.dask[key]))

In [ ]:
print_dask(i_2)

In [ ]:
print_dask(d_i_2)

## Delayed.visualize()

It's kinda annoying that we have to write a special function to see what the graph looks like!

Fortunately, there's a better way!  Use the `visualize` method of the `Delayed` object.

In [ ]:
i_2.visualize()

In [ ]:
d_i_2.visualize()

### Notice!

If we visualize the persisted versions of these `Delayed` objects, what do you get?

In [ ]:
i_2_p.visualize()

In [ ]:
d_i_2_p.visualize()

The first objects in the Task Graphs are *data*, now!  Before, they were function calls!